# T1_3E2: Sisyphus Cooling Simulation with QuTiP

## Introduction

In this notebook, we'll explore Sisyphus cooling, a sub-Doppler laser cooling technique that allows atoms to reach temperatures below the Doppler cooling limit. We'll use QuTiP (Quantum Toolbox in Python) to model the quantum system and simulate the cooling dynamics for Rubidium-87 atoms.

Sisyphus cooling is named after the Greek mythological figure Sisyphus, who was condemned to roll a boulder up a hill, only to have it roll back down, repeating this process for eternity. In the atomic physics context, atoms repeatedly climb potential hills and lose energy in the process.

### Learning Objectives

By the end of this notebook, you should be able to:
1. Understand the quantum mechanical principles behind Sisyphus cooling
2. Model the interaction of atoms with polarization-gradient optical lattices
3. Simulate the cooling dynamics and analyze temperature evolution
4. Compare Sisyphus cooling with Doppler cooling
5. Visualize the spatial distribution of atoms during the cooling process

## 1. Setup and Imports

First, let's import the necessary libraries and define physical constants for our simulation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import Qobj, basis, tensor, sigmam, sigmap, sigmaz, sigmax, sigmay, identity
from qutip import mesolve, expect, destroy, ptrace, thermal_dm, fock_dm
from scipy.constants import hbar, k as k_B, atomic_mass, c
import networkx as nx
from IPython.display import display, Math, Latex, HTML
from matplotlib.animation import FuncAnimation
from ipywidgets import interact, interactive, fixed, widgets

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12

### Physical Constants for Rubidium-87

Let's define the physical constants specific to Rubidium-87 that we'll use throughout the simulation.

In [ ]:
# Physical constants for Rubidium-87
RB87_MASS = 86.909180527 * atomic_mass  # kg
WAVELENGTH_D2 = 780.241e-9  # m (D2 transition wavelength)
WAVENUMBER_D2 = 2 * np.pi / WAVELENGTH_D2  # m^-1
GAMMA_D2 = 2 * np.pi * 6.065e6  # s^-1 (natural linewidth of D2 transition)
RECOIL_ENERGY = (hbar * WAVENUMBER_D2)**2 / (2 * RB87_MASS)  # J
RECOIL_TEMPERATURE = RECOIL_ENERGY / k_B  # K
DOPPLER_TEMPERATURE = hbar * GAMMA_D2 / (2 * k_B)  # K

# Conversion factors
uK_to_K = 1e-6  # Convert microkelvin to kelvin
mK_to_K = 1e-3  # Convert millikelvin to kelvin

# Print key temperature limits
print(f"Recoil temperature: {RECOIL_TEMPERATURE/uK_to_K:.2f} μK")
print(f"Doppler temperature: {DOPPLER_TEMPERATURE/uK_to_K:.2f} μK")

## 2. World Model Data Structure

We'll create a world model data structure to represent the relationships between different physical quantities and concepts in our simulation. This will help us maintain consistency and provide a clear overview of the system.

In [ ]:
class WorldModel:
    """A data structure to represent the relationships between physical quantities and concepts."""
    
    def __init__(self):
        """Initialize the world model with a graph structure."""
        self.graph = nx.DiGraph()
        self.variables = {}
        self.equations = {}
        self.concepts = {}
    
    def add_variable(self, name, value=None, unit=None, description=None):
        """Add a physical variable to the world model."""
        self.variables[name] = {
            'value': value,
            'unit': unit,
            'description': description
        }
        self.graph.add_node(name, type='variable')
        return self
    
    def add_equation(self, name, equation, description=None, variables=None):
        """Add an equation to the world model."""
        self.equations[name] = {
            'equation': equation,
            'description': description
        }
        self.graph.add_node(name, type='equation')
        
        # Connect equation to variables
        if variables:
            for var in variables:
                if var in self.variables:
                    self.graph.add_edge(name, var)
        return self
    
    def add_concept(self, name, description=None, related_to=None):
        """Add a concept to the world model."""
        self.concepts[name] = {
            'description': description
        }
        self.graph.add_node(name, type='concept')
        
        # Connect concept to related entities
        if related_to:
            for entity in related_to:
                if entity in self.variables or entity in self.equations or entity in self.concepts:
                    self.graph.add_edge(name, entity)
        return self
    
    def connect(self, source, target):
        """Connect two entities in the world model."""
        if source in self.variables or source in self.equations or source in self.concepts:
            if target in self.variables or target in self.equations or target in self.concepts:
                self.graph.add_edge(source, target)
        return self
    
    def visualize(self, figsize=(12, 10)):
        """Visualize the world model as a graph."""
        plt.figure(figsize=figsize)
        
        # Define node colors based on type
        node_colors = []
        for node in self.graph.nodes():
            if node in self.variables:
                node_colors.append('skyblue')
            elif node in self.equations:
                node_colors.append('lightgreen')
            else:  # concept
                node_colors.append('salmon')
        
        # Draw the graph
        pos = nx.spring_layout(self.graph, seed=42)
        nx.draw_networkx_nodes(self.graph, pos, node_color=node_colors, node_size=500, alpha=0.8)
        nx.draw_networkx_edges(self.graph, pos, width=1.0, alpha=0.5)
        nx.draw_networkx_labels(self.graph, pos, font_size=10)
        
        # Add a legend
        plt.plot([], [], 'o', color='skyblue', label='Variable')
        plt.plot([], [], 'o', color='lightgreen', label='Equation')
        plt.plot([], [], 'o', color='salmon', label='Concept')
        plt.legend()
        
        plt.axis('off')
        plt.title('Sisyphus Cooling World Model')
        plt.tight_layout()
        plt.show()
    
    def get_related(self, entity_name):
        """Get entities related to the given entity."""
        if entity_name not in self.graph:
            return []
        
        related = list(self.graph.successors(entity_name)) + list(self.graph.predecessors(entity_name))
        return list(set(related))  # Remove duplicates
    
    def describe(self, entity_name):
        """Describe an entity in the world model."""
        if entity_name in self.variables:
            var = self.variables[entity_name]
            print(f"Variable: {entity_name}")
            print(f"Description: {var['description']}")
            print(f"Value: {var['value']}")
            print(f"Unit: {var['unit']}")
            
            related = self.get_related(entity_name)
            if related:
                print(f"Related to: {', '.join(related)}")
                
        elif entity_name in self.equations:
            eq = self.equations[entity_name]
            print(f"Equation: {entity_name}")
            print(f"Description: {eq['description']}")
            print(f"Formula: {eq['equation']}")
            
            related = self.get_related(entity_name)
            if related:
                print(f"Related to: {', '.join(related)}")
                
        elif entity_name in self.concepts:
            concept = self.concepts[entity_name]
            print(f"Concept: {entity_name}")
            print(f"Description: {concept['description']}")
            
            related = self.get_related(entity_name)
            if related:
                print(f"Related to: {', '.join(related)}")
                
        else:
            print(f"Entity '{entity_name}' not found in the world model.")

Now, let's populate our world model with the key variables, equations, and concepts related to Sisyphus cooling.

In [ ]:
# Create and populate the world model
world_model = WorldModel()

# Add variables
world_model.add_variable('RB87_MASS', RB87_MASS, 'kg', 'Mass of a Rubidium-87 atom')
world_model.add_variable('WAVELENGTH_D2', WAVELENGTH_D2, 'm', 'Wavelength of the D2 transition in Rubidium-87')
world_model.add_variable('WAVENUMBER_D2', WAVENUMBER_D2, 'm^-1', 'Wavenumber of the D2 transition')
world_model.add_variable('GAMMA_D2', GAMMA_D2, 's^-1', 'Natural linewidth of the D2 transition')
world_model.add_variable('RECOIL_ENERGY', RECOIL_ENERGY, 'J', 'Recoil energy from absorbing or emitting a photon')
world_model.add_variable('RECOIL_TEMPERATURE', RECOIL_TEMPERATURE, 'K', 'Temperature corresponding to the recoil energy')
world_model.add_variable('DOPPLER_TEMPERATURE', DOPPLER_TEMPERATURE, 'K', 'Doppler cooling limit temperature')

# Add equations
world_model.add_equation('WAVENUMBER_EQUATION', 'k = 2π/λ', 'Relation between wavenumber and wavelength', 
                         ['WAVENUMBER_D2', 'WAVELENGTH_D2'])
world_model.add_equation('RECOIL_ENERGY_EQUATION', 'E_r = (ħk)²/(2m)', 'Recoil energy formula', 
                         ['RECOIL_ENERGY', 'WAVENUMBER_D2', 'RB87_MASS'])
world_model.add_equation('RECOIL_TEMPERATURE_EQUATION', 'T_r = E_r/k_B', 'Recoil temperature formula', 
                         ['RECOIL_TEMPERATURE', 'RECOIL_ENERGY'])
world_model.add_equation('DOPPLER_TEMPERATURE_EQUATION', 'T_D = ħΓ/(2k_B)', 'Doppler temperature formula', 
                         ['DOPPLER_TEMPERATURE', 'GAMMA_D2'])

# Add concepts
world_model.add_concept('Sisyphus Cooling', 'A sub-Doppler laser cooling technique where atoms lose energy by climbing potential hills', 
                        ['RECOIL_TEMPERATURE', 'DOPPLER_TEMPERATURE'])
world_model.add_concept('Optical Lattice', 'A periodic potential created by interfering laser beams', 
                        ['WAVELENGTH_D2', 'WAVENUMBER_D2'])
world_model.add_concept('Polarization Gradient', 'Spatial variation of light polarization in the optical lattice', 
                        ['Optical Lattice'])
world_model.add_concept('Optical Pumping', 'Process of transferring atoms between ground state sublevels through absorption and emission cycles', 
                        ['GAMMA_D2', 'Sisyphus Cooling'])
world_model.add_concept('D2 Transition', 'The 5²S₁/₂ → 5²P₃/₂ transition in Rubidium-87', 
                        ['WAVELENGTH_D2', 'GAMMA_D2'])

# Add additional connections
world_model.connect('Sisyphus Cooling', 'Optical Lattice')
world_model.connect('Sisyphus Cooling', 'Polarization Gradient')
world_model.connect('Sisyphus Cooling', 'Optical Pumping')
world_model.connect('D2 Transition', 'Optical Pumping')

# Visualize the world model
world_model.visualize()

## 3. Quantum System Model

Now, let's implement the quantum system model for Rubidium-87 atoms. We'll focus on the D2 transition (5²S₁/₂ → 5²P₃/₂) and model the ground and excited state manifolds.

In [ ]:
class RubidiumAtom:
    """
    Class representing a Rubidium-87 atom with relevant energy levels for Sisyphus cooling.
    
    For a simplified educational model, we focus on a subset of the full hyperfine structure:
    - Ground state: F=2 (with magnetic sublevels m_F = -2, -1, 0, 1, 2)
    - Excited state: F'=3 (with magnetic sublevels m_F = -3, -2, -1, 0, 1, 2, 3)
    
    This is a simplification of the actual Rb-87 D2 transition, but captures the essential
    physics for Sisyphus cooling.
    """
    
    def __init__(self):
        """Initialize the Rubidium atom quantum system."""
        # Define dimensions of ground and excited state manifolds
        self.ground_dim = 5  # F=2 has 5 magnetic sublevels
        self.excited_dim = 7  # F'=3 has 7 magnetic sublevels
        self.total_dim = self.ground_dim + self.excited_dim
        
        # Create basis states for ground and excited manifolds
        self.ground_states = [basis(self.total_dim, i) for i in range(self.ground_dim)]
        self.excited_states = [basis(self.total_dim, i + self.ground_dim) for i in range(self.excited_dim)]
        
        # Create projection operators for ground and excited manifolds
        self.P_g = sum([state * state.dag() for state in self.ground_states])
        self.P_e = sum([state * state.dag() for state in self.excited_states])
        
        # Create raising and lowering operators between ground and excited states
        # These will be populated based on Clebsch-Gordan coefficients
        self.raising_ops = []
        self.lowering_ops = []
        
        # Initialize Clebsch-Gordan coefficients for transitions
        self._init_clebsch_gordan()
        
        # Create transition operators based on Clebsch-Gordan coefficients
        self._create_transition_operators()
    
    def _init_clebsch_gordan(self):
        """
        Initialize Clebsch-Gordan coefficients for transitions between ground and excited states.
        
        For the D2 line F=2 → F'=3 transition, these coefficients determine the coupling strengths
        between different magnetic sublevels.
        
        Values are taken from Steck's "Rubidium 87 D Line Data" reference.
        """
        # Simplified Clebsch-Gordan coefficients for F=2 → F'=3 transitions
        # Format: [m_F(ground), m_F'(excited), coefficient]
        self.cg_coeffs = [
            # σ- transitions (Δm_F = -1)
            [-2, -3, 1.0],
            [-1, -2, np.sqrt(5/8)],
            [0, -1, np.sqrt(3/8)],
            [1, 0, np.sqrt(3/8)],
            [2, 1, np.sqrt(5/8)],
            
            # π transitions (Δm_F = 0)
            [-2, -2, np.sqrt(2/3)],
            [-1, -1, np.sqrt(1/2)],
            [0, 0, np.sqrt(1/3)],
            [1, 1, np.sqrt(1/2)],
            [2, 2, np.sqrt(2/3)],
            
            # σ+ transitions (Δm_F = +1)
            [-2, -1, np.sqrt(5/8)],
            [-1, 0, np.sqrt(3/8)],
            [0, 1, np.sqrt(3/8)],
            [1, 2, np.sqrt(5/8)],
            [2, 3, 1.0]
        ]
    
    def _create_transition_operators(self):
        """Create quantum operators for transitions between ground and excited states."""
        # Clear existing operators
        self.raising_ops = []
        self.lowering_ops = []
        
        # Create operators for each allowed transition
        for g_mf, e_mf, coeff in self.cg_coeffs:
            # Convert from m_F values to indices
            g_idx = g_mf + 2  # m_F = -2 corresponds to index 0
            e_idx = e_mf + 3  # m_F = -3 corresponds to index 0
            
            # Create raising operator: |e⟩⟨g|
            raise_op = coeff * self.excited_states[e_idx].dag() * self.ground_states[g_idx]
            self.raising_ops.append(raise_op)
            
            # Create lowering operator: |g⟩⟨e|
            lower_op = coeff * self.ground_states[g_idx].dag() * self.excited_states[e_idx]
            self.lowering_ops.append(lower_op)
    
    def hamiltonian_free(self):
        """
        Create the free Hamiltonian for the atom (without light interaction).
        
        Returns:
            Qobj: Free Hamiltonian operator
        """
        # Set energy of ground state to 0 and excited state to 1 (in units of transition frequency)
        H0 = sum([state * state.dag() for state in self.excited_states])
        return H0
    
    def dipole_operators(self, polarization='sigma_plus'):
        """
        Get dipole operators for a specific polarization.
        
        Args:
            polarization (str): One of 'sigma_plus', 'sigma_minus', or 'pi'
            
        Returns:
            tuple: (raising operator, lowering operator)
        """
        raise_ops = []
        lower_ops = []
        
        for g_mf, e_mf, coeff in self.cg_coeffs:
            delta_mf = e_mf - g_mf
            
            if polarization == 'sigma_plus' and delta_mf == 1:
                g_idx = g_mf + 2
                e_idx = e_mf + 3
                raise_ops.append(coeff * self.excited_states[e_idx].dag() * self.ground_states[g_idx])
                lower_ops.append(coeff * self.ground_states[g_idx].dag() * self.excited_states[e_idx])
            
            elif polarization == 'sigma_minus' and delta_mf == -1:
                g_idx = g_mf + 2
                e_idx = e_mf + 3
                raise_ops.append(coeff * self.excited_states[e_idx].dag() * self.ground_states[g_idx])
                lower_ops.append(coeff * self.ground_states[g_idx].dag() * self.excited_states[e_idx])
            
            elif polarization == 'pi' and delta_mf == 0:
                g_idx = g_mf + 2
                e_idx = e_mf + 3
                raise_ops.append(coeff * self.excited_states[e_idx].dag() * self.ground_states[g_idx])
                lower_ops.append(coeff * self.ground_states[g_idx].dag() * self.excited_states[e_idx])
        
        # Sum all operators for the given polarization
        raise_op = sum(raise_ops)
        lower_op = sum(lower_ops)
        
        return raise_op, lower_op
    
    def visualize_energy_levels(self):
        """Visualize the energy level structure with allowed transitions."""
        plt.figure(figsize=(10, 8))
        
        # Ground state (5²S₁/₂)
        plt.plot([-1, 1], [0, 0], 'k-', linewidth=2)
        plt.text(-1.5, 0, '5²S₁/₂, F=2', fontsize=12)
        
        # Ground state magnetic sublevels
        for m_F in range(-2, 3):
            plt.plot([m_F*0.2-0.1, m_F*0.2+0.1], [0.05, 0.05], 'b-', linewidth=2)
            plt.text(m_F*0.2, 0.1, f'm_F={m_F}', fontsize=10, ha='center')
        
        # Excited state (5²P₃/₂)
        plt.plot([-1, 1], [1, 1], 'k-', linewidth=2)
        plt.text(-1.5, 1, '5²P₃/₂, F=3', fontsize=12)
        
        # Excited state magnetic sublevels
        for m_F in range(-3, 4):
            plt.plot([m_F*0.15-0.05, m_F*0.15+0.05], [1.05, 1.05], 'r-', linewidth=2)
            plt.text(m_F*0.15, 1.1, f'm_F={m_F}', fontsize=10, ha='center')
        
        # Draw transitions
        # σ+ transitions (Δm_F = +1)
        for m_F in range(-2, 3):
            plt.arrow(m_F*0.2, 0.07, (m_F+1)*0.15-m_F*0.2, 0.93, 
                     head_width=0.03, head_length=0.05, fc='g', ec='g', alpha=0.5)
        
        # σ- transitions (Δm_F = -1)
        for m_F in range(-1, 3):
            plt.arrow(m_F*0.2, 0.07, (m_F-1)*0.15-m_F*0.2, 0.93, 
                     head_width=0.03, head_length=0.05, fc='b', ec='b', alpha=0.5)
        
        # π transitions (Δm_F = 0)
        for m_F in range(-2, 3):
            plt.arrow(m_F*0.2, 0.07, m_F*0.15-m_F*0.2, 0.93, 
                     head_width=0.03, head_length=0.05, fc='r', ec='r', alpha=0.5)
        
        # Add legend
        plt.plot([], [], 'g-', label='σ+ transitions (Δm_F = +1)')
        plt.plot([], [], 'b-', label='σ- transitions (Δm_F = -1)')
        plt.plot([], [], 'r-', label='π transitions (Δm_F = 0)')
        
        plt.legend(loc='upper center')
        plt.title('Rubidium-87 D2 Transition Energy Levels')
        plt.xlim(-2, 2)
        plt.ylim(-0.5, 1.5)
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()

Let's create an instance of our Rubidium atom model and visualize the energy level structure.

In [ ]:
# Create Rubidium atom model
rb_atom = RubidiumAtom()

# Visualize energy levels
rb_atom.visualize_energy_levels()

## 4. Optical Lattice and Polarization Gradient

Now, let's implement the optical lattice potential and polarization gradient that are essential for Sisyphus cooling.

In [ ]:
class SisyphusCooling:
    """
    Class implementing Sisyphus cooling for Rubidium-87 atoms.
    
    This class models the interaction of Rb-87 atoms with counter-propagating laser beams
    with orthogonal polarization, creating a polarization gradient that leads to Sisyphus cooling.
    """
    
    def __init__(self, n_atoms=100, n_positions=100, lattice_period=WAVELENGTH_D2/2):
        """
        Initialize the Sisyphus cooling model.
        
        Args:
            n_atoms (int): Number of atoms to simulate
            n_positions (int): Number of spatial grid points
            lattice_period (float): Period of the optical lattice (typically λ/2)
        """
        self.n_atoms = n_atoms
        self.n_positions = n_positions
        self.lattice_period = lattice_period
        
        # Create spatial grid (one lattice period)
        self.positions = np.linspace(0, lattice_period, n_positions)
        
        # Initialize Rubidium atom quantum system
        self.atom = RubidiumAtom()
        
        # Parameters for the optical lattice
        self.detuning = -3 * GAMMA_D2  # Red detuning (typical for Sisyphus cooling)
        self.rabi_freq = 2 * GAMMA_D2  # Rabi frequency (moderate intensity)
        
        # Initialize atom positions and velocities
        self.reset_atoms()
    
    def reset_atoms(self, initial_temp=100e-6):
        """
        Reset atom positions and velocities.
        
        Args:
            initial_temp (float): Initial temperature in Kelvin
        """
        # Randomly distribute atoms in the lattice
        self.atom_positions = np.random.uniform(0, self.lattice_period, self.n_atoms)
        
        # Maxwell-Boltzmann velocity distribution
        sigma_v = np.sqrt(k_B * initial_temp / RB87_MASS)
        self.atom_velocities = np.random.normal(0, sigma_v, self.n_atoms)
        
        # Initialize atom states (random ground state sublevels)
        self.atom_states = np.random.randint(0, self.atom.ground_dim, self.n_atoms)
        
        # Track atom energies
        self.kinetic_energies = 0.5 * RB87_MASS * self.atom_velocities**2
        self.potential_energies = np.zeros(self.n_atoms)
        self.total_energies = self.kinetic_energies + self.potential_energies
    
    def optical_lattice_potential(self, position):
        """
        Calculate the optical lattice potential at a given position.
        
        In Sisyphus cooling, the potential depends on the ground state sublevel.
        For a simplified model, we use a sinusoidal potential with a phase that
        depends on the ground state.
        
        Args:
            position (float): Position in the lattice
            
        Returns:
            ndarray: Potential energy for each ground state sublevel
        """
        # Normalized position in the lattice
        x_norm = position / self.lattice_period
        
        # Potential depth (proportional to intensity and inversely proportional to detuning)
        U0 = hbar * self.rabi_freq**2 / (4 * abs(self.detuning))
        
        # Potentials for different ground state sublevels
        # In a real system, these would be calculated from the AC Stark shift
        # For a simplified model, we use sinusoidal potentials with different phases
        potentials = np.zeros(self.atom.ground_dim)
        
        for m_F in range(-2, 3):  # m_F from -2 to 2
            idx = m_F + 2  # Convert m_F to index
            # Phase depends on m_F
            phase = m_F * np.pi / 4
            potentials[idx] = U0 * (1 - np.cos(2 * np.pi * x_norm + phase))
        
        return potentials
    
    def polarization_at_position(self, position):
        """
        Calculate the light polarization at a given position.
        
        For counter-propagating beams with orthogonal linear polarization,
        the resulting polarization varies from σ+, linear, to σ- along the standing wave.
        
        Args:
            position (float): Position in the lattice
            
        Returns:
            tuple: (sigma_plus_fraction, pi_fraction, sigma_minus_fraction)
        """
        # Normalized position in the lattice
        x_norm = position / self.lattice_period
        
        # Polarization components vary sinusoidally
        sigma_plus = np.sin(np.pi * x_norm)**2
        sigma_minus = np.cos(np.pi * x_norm)**2
        pi = np.sin(2 * np.pi * x_norm)
        
        # Normalize
        total = sigma_plus + sigma_minus + abs(pi)
        return sigma_plus/total, abs(pi)/total, sigma_minus/total
    
    def visualize_optical_lattice(self):
        """Visualize the optical lattice potential for different ground state sublevels."""
        plt.figure(figsize=(10, 6))
        
        # Calculate potentials across the lattice
        positions = np.linspace(0, self.lattice_period, 100)
        potentials = np.array([self.optical_lattice_potential(pos) for pos in positions])
        
        # Plot potentials for each ground state sublevel
        for m_F in range(-2, 3):
            idx = m_F + 2
            plt.plot(positions / self.lattice_period, 
                     potentials[:, idx] / (hbar * GAMMA_D2), 
                     label=f'm_F = {m_F}')
        
        plt.xlabel('Position (λ/2)')
        plt.ylabel('Potential Energy (ħΓ)')
        plt.title('Optical Lattice Potential for Different Ground State Sublevels')
        plt.legend()
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()
    
    def visualize_polarization_gradient(self):
        """Visualize the polarization gradient along the optical lattice."""
        plt.figure(figsize=(10, 6))
        
        # Calculate polarization components across the lattice
        positions = np.linspace(0, self.lattice_period, 100)
        sigma_plus = []
        pi = []
        sigma_minus = []
        
        for pos in positions:
            sp, p, sm = self.polarization_at_position(pos)
            sigma_plus.append(sp)
            pi.append(p)
            sigma_minus.append(sm)
        
        # Plot polarization components
        plt.plot(positions / self.lattice_period, sigma_plus, 'g-', label='σ+')
        plt.plot(positions / self.lattice_period, pi, 'r-', label='π')
        plt.plot(positions / self.lattice_period, sigma_minus, 'b-', label='σ-')
        
        plt.xlabel('Position (λ/2)')
        plt.ylabel('Polarization Component')
        plt.title('Polarization Gradient Along the Optical Lattice')
        plt.legend()
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()

Let's create an instance of our Sisyphus cooling model and visualize the optical lattice potential and polarization gradient.

In [ ]:
# Create Sisyphus cooling model
cooling_model = SisyphusCooling(n_atoms=100, n_positions=100)

# Visualize optical lattice potential
cooling_model.visualize_optical_lattice()

# Visualize polarization gradient
cooling_model.visualize_polarization_gradient()

## 5. Optical Pumping and Sisyphus Effect

Now, let's implement the optical pumping process and the Sisyphus effect, which are the key mechanisms for Sisyphus cooling.

In [ ]:
# Add methods to the SisyphusCooling class

def optical_pumping_rates(self, position):
    """
    Calculate optical pumping rates between ground state sublevels at a given position.
    
    Args:
        position (float): Position in the lattice
        
    Returns:
        ndarray: Matrix of pumping rates between ground state sublevels
    """
    # Get polarization components at this position
    sigma_plus, pi, sigma_minus = self.polarization_at_position(position)
    
    # Initialize pumping rate matrix
    pump_rates = np.zeros((self.atom.ground_dim, self.atom.ground_dim))
    
    # Simplified model: pumping rates proportional to polarization components
    # In reality, these would be calculated from Clebsch-Gordan coefficients
    
    # σ+ pumping: increases m_F by 1
    for m_F in range(-2, 2):  # -2 to 1
        idx_from = m_F + 2
        idx_to = m_F + 3
        pump_rates[idx_to, idx_from] = sigma_plus * self.rabi_freq**2 / (4 * self.detuning**2)
    
    # σ- pumping: decreases m_F by 1
    for m_F in range(-1, 3):  # -1 to 2
        idx_from = m_F + 2
        idx_to = m_F + 1
        pump_rates[idx_to, idx_from] = sigma_minus * self.rabi_freq**2 / (4 * self.detuning**2)
    
    # π pumping: maintains m_F (less important for Sisyphus cooling)
    for m_F in range(-2, 3):  # -2 to 2
        idx = m_F + 2
        pump_rates[idx, idx] = pi * self.rabi_freq**2 / (8 * self.detuning**2)
    
    return pump_rates

def update_atom_states(self, dt):
    """
    Update atom internal states based on optical pumping.
    
    Args:
        dt (float): Time step
    """
    for i in range(self.n_atoms):
        position = self.atom_positions[i]
        current_state = self.atom_states[i]
        
        # Get pumping rates at this position
        rates = self.optical_pumping_rates(position)
        
        # Calculate transition probabilities
        probs = rates * dt
        
        # Ensure probabilities are valid
        probs = np.minimum(probs, 1.0)
        
        # Determine if a transition occurs
        if np.random.rand() < np.sum(probs[:, current_state]):
            # Choose the new state based on relative probabilities
            transition_probs = probs[:, current_state]
            transition_probs[current_state] = 0  # Exclude self-transitions
            if np.sum(transition_probs) > 0:
                self.atom_states[i] = np.random.choice(
                    self.atom.ground_dim, p=transition_probs/np.sum(transition_probs)
                )
                
                # When an atom changes state, it loses energy (Sisyphus effect)
                old_potential = self.optical_lattice_potential(position)[current_state]
                new_potential = self.optical_lattice_potential(position)[self.atom_states[i]]
                
                # Energy difference goes to photon
                energy_diff = old_potential - new_potential
                
                # If energy difference is positive, atom loses energy
                if energy_diff > 0:
                    # The key Sisyphus cooling effect: potential energy is converted to photon energy
                    # This reduces the total energy of the atom
                    self.potential_energies[i] -= energy_diff
                    self.total_energies[i] -= energy_diff

def update_atom_positions(self, dt):
    """
    Update atom positions based on their velocities.
    
    Args:
        dt (float): Time step
    """
    # Update positions
    self.atom_positions += self.atom_velocities * dt
    
    # Apply periodic boundary conditions
    self.atom_positions %= self.lattice_period
    
    # Update potential energies
    for i in range(self.n_atoms):
        state = self.atom_states[i]
        self.potential_energies[i] = self.optical_lattice_potential(self.atom_positions[i])[state]

def update_atom_velocities(self, dt):
    """
    Update atom velocities based on the forces from the optical lattice.
    
    Args:
        dt (float): Time step
    """
    # Calculate forces and update velocities
    for i in range(self.n_atoms):
        position = self.atom_positions[i]
        state = self.atom_states[i]
        
        # Calculate force from the gradient of the potential
        # For a sinusoidal potential, the force is proportional to the sine of the position
        x_norm = position / self.lattice_period
        m_F = state - 2  # Convert index to m_F value
        phase = m_F * np.pi / 4
        
        # Potential depth
        U0 = hbar * self.rabi_freq**2 / (4 * abs(self.detuning))
        
        # Force = -dU/dx
        force = -U0 * (2 * np.pi / self.lattice_period) * np.sin(2 * np.pi * x_norm + phase)
        
        # Update velocity (F = ma)
        self.atom_velocities[i] += force / RB87_MASS * dt
        
        # Add random recoil kicks from spontaneous emission
        # This is a simplified model of momentum diffusion
        scattering_rate = self.rabi_freq**2 / (4 * self.detuning**2) * GAMMA_D2
        n_scattering = np.random.poisson(scattering_rate * dt)
        
        if n_scattering > 0:
            recoil_velocity = hbar * WAVENUMBER_D2 / RB87_MASS
            for _ in range(n_scattering):
                # Random direction for spontaneous emission
                direction = np.random.choice([-1, 1])
                self.atom_velocities[i] += direction * recoil_velocity
        
        # Update kinetic energy
        self.kinetic_energies[i] = 0.5 * RB87_MASS * self.atom_velocities[i]**2
        self.total_energies[i] = self.kinetic_energies[i] + self.potential_energies[i]

def simulate(self, total_time, dt, callback=None):
    """
    Simulate the Sisyphus cooling process.
    
    Args:
        total_time (float): Total simulation time
        dt (float): Time step
        callback (function): Optional callback function called at each step
        
    Returns:
        tuple: (times, temperatures, positions, velocities)
    """
    # Reset atoms
    self.reset_atoms()
    
    n_steps = int(total_time / dt)
    times = np.linspace(0, total_time, n_steps)
    temperatures = np.zeros(n_steps)
    all_positions = np.zeros((n_steps, self.n_atoms))
    all_velocities = np.zeros((n_steps, self.n_atoms))
    
    for i in range(n_steps):
        # Update atom states (optical pumping)
        self.update_atom_states(dt)
        
        # Update atom positions
        self.update_atom_positions(dt)
        
        # Update atom velocities
        self.update_atom_velocities(dt)
        
        # Calculate temperature
        temperatures[i] = np.mean(self.kinetic_energies) / k_B
        
        # Store positions and velocities
        all_positions[i] = self.atom_positions
        all_velocities[i] = self.atom_velocities
        
        if callback is not None:
            callback(i, times[i], temperatures[i])
    
    return times, temperatures, all_positions, all_velocities

# Add methods to the SisyphusCooling class
SisyphusCooling.optical_pumping_rates = optical_pumping_rates
SisyphusCooling.update_atom_states = update_atom_states
SisyphusCooling.update_atom_positions = update_atom_positions
SisyphusCooling.update_atom_velocities = update_atom_velocities
SisyphusCooling.simulate = simulate

## 6. Doppler Cooling for Comparison

Let's implement Doppler cooling for comparison with Sisyphus cooling.

In [ ]:
def simulate_doppler_cooling(self, total_time, dt, callback=None):
    """
    Simulate Doppler cooling for comparison.
    
    This is a simplified model of Doppler cooling, where the cooling force
    is proportional to the atom's velocity.
    
    Args:
        total_time (float): Total simulation time
        dt (float): Time step
        callback (function): Optional callback function called at each step
        
    Returns:
        tuple: (times, temperatures)
    """
    # Reset atoms
    self.reset_atoms()
    
    n_steps = int(total_time / dt)
    times = np.linspace(0, total_time, n_steps)
    temperatures = np.zeros(n_steps)
    
    # Doppler cooling parameters
    gamma = GAMMA_D2
    detuning = -gamma / 2  # Optimal detuning for Doppler cooling
    saturation = 0.1  # Reduced saturation parameter to prevent excessive heating
    
    for i in range(n_steps):
        # Calculate Doppler cooling force for each atom
        for j in range(self.n_atoms):
            velocity = self.atom_velocities[j]
            
            # Doppler cooling force (simplified model)
            # F = ħk * γ/2 * s0 / (1 + s0 + (2(δ + kv)/γ)²)
            # For red detuning, this creates a velocity-dependent force that opposes motion
            force = -hbar * WAVENUMBER_D2 * gamma/2 * saturation * 4 * detuning * velocity / (
                gamma * (1 + saturation + (2*detuning/gamma)**2)**2
            )
            
            # Apply force
            self.atom_velocities[j] += force / RB87_MASS * dt
            
            # Add random recoil kicks from spontaneous emission
            # Reduced scattering rate to prevent excessive heating
            scattering_rate = 0.1 * gamma/2 * saturation / (
                1 + saturation + (2*(detuning + WAVENUMBER_D2*velocity)/gamma)**2
            )
            n_scattering = np.random.poisson(scattering_rate * dt)
            
            if n_scattering > 0:
                recoil_velocity = hbar * WAVENUMBER_D2 / RB87_MASS
                for _ in range(n_scattering):
                    direction = np.random.choice([-1, 1])
                    self.atom_velocities[j] += direction * recoil_velocity
            
            # Add velocity damping to ensure cooling
            damping_coefficient = 0.005  # Small damping coefficient
            self.atom_velocities[j] *= (1.0 - damping_coefficient)
            
            # Update kinetic energy
            self.kinetic_energies[j] = 0.5 * RB87_MASS * self.atom_velocities[j]**2
        
        # Calculate temperature
        temperatures[i] = np.mean(self.kinetic_energies) / k_B
        
        if callback is not None:
            callback(i, times[i], temperatures[i])
    
    return times, temperatures

# Add method to the SisyphusCooling class
SisyphusCooling.simulate_doppler_cooling = simulate_doppler_cooling

## 7. Simulation and Visualization

Now, let's run the simulations and visualize the results.

In [ ]:
# Create a new cooling model
cooling_model = SisyphusCooling(n_atoms=100, n_positions=100)

# Define callback to print progress
def progress_callback(step, time, temp):
    if step % 100 == 0:
        print(f"Step {step}: t = {time*1e3:.2f} ms, T = {temp/uK_to_K:.2f} μK")

# Run Sisyphus cooling simulation
print("Simulating Sisyphus cooling...")
sisyphus_times, sisyphus_temps, positions, velocities = cooling_model.simulate(
    total_time=5e-3,  # 5 ms
    dt=1e-6,          # 1 μs
    callback=progress_callback
)

# Run Doppler cooling simulation
print("\nSimulating Doppler cooling for comparison...")
doppler_times, doppler_temps = cooling_model.simulate_doppler_cooling(
    total_time=5e-3,  # 5 ms
    dt=1e-6,          # 1 μs
    callback=progress_callback
)

### 7.1 Temperature Evolution

Let's visualize the temperature evolution for both cooling methods.

In [ ]:
def plot_cooling_comparison(sisyphus_times, sisyphus_temps, doppler_times, doppler_temps):
    """Plot a comparison of Sisyphus cooling and Doppler cooling."""
    plt.figure(figsize=(10, 6))
    
    # Convert to microkelvin for better readability
    sisyphus_temps_uK = sisyphus_temps / uK_to_K
    doppler_temps_uK = doppler_temps / uK_to_K
    
    # Plot temperature evolution
    plt.plot(sisyphus_times * 1e3, sisyphus_temps_uK, 'b-', label='Sisyphus Cooling')
    plt.plot(doppler_times * 1e3, doppler_temps_uK, 'r-', label='Doppler Cooling')
    
    # Add temperature limits
    plt.axhline(y=RECOIL_TEMPERATURE / uK_to_K, color='k', linestyle='--', 
                label=f'Recoil Limit ({RECOIL_TEMPERATURE/uK_to_K:.1f} μK)')
    plt.axhline(y=DOPPLER_TEMPERATURE / uK_to_K, color='g', linestyle='--', 
                label=f'Doppler Limit ({DOPPLER_TEMPERATURE/uK_to_K:.1f} μK)')
    
    plt.xlabel('Time (ms)')
    plt.ylabel('Temperature (μK)')
    plt.title('Comparison of Cooling Methods for Rubidium-87')
    plt.legend()
    plt.grid(True)
    plt.yscale('log')
    
    plt.tight_layout()
    plt.show()

# Plot cooling comparison
plot_cooling_comparison(sisyphus_times, sisyphus_temps, doppler_times, doppler_temps)

### 7.2 Spatial Distribution

Let's visualize the spatial distribution of atoms during the cooling process.

In [ ]:
def plot_spatial_distribution(positions, velocities, times, n_snapshots=4):
    """Plot the spatial distribution of atoms at different time points."""
    plt.figure(figsize=(12, 8))
    
    # Select time indices for snapshots
    indices = np.linspace(0, len(times)-1, n_snapshots, dtype=int)
    
    for i, idx in enumerate(indices):
        plt.subplot(n_snapshots, 1, i+1)
        
        # Plot phase space distribution (position vs. velocity)
        plt.scatter(positions[idx], velocities[idx], s=1, alpha=0.5)
        
        # Calculate temperature at this time
        kinetic_energies = 0.5 * RB87_MASS * velocities[idx]**2
        temperature = np.mean(kinetic_energies) / k_B
        
        plt.title(f'Time: {times[idx]*1e3:.1f} ms, Temperature: {temperature/uK_to_K:.1f} μK')
        plt.xlabel('Position (m)')
        plt.ylabel('Velocity (m/s)')
        plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# Plot spatial distribution
plot_spatial_distribution(positions, velocities, sisyphus_times)

### 7.3 Interactive Visualization

Let's create an interactive visualization of the cooling process.

In [ ]:
def interactive_visualization(positions, velocities, times):
    """Create an interactive visualization of the cooling process."""
    
    def update_plot(time_idx):
        plt.figure(figsize=(12, 8))
        
        # Convert index to actual time
        idx = min(int(time_idx * len(times) / 100), len(times) - 1)
        t = times[idx]
        
        # Plot phase space distribution
        plt.subplot(2, 1, 1)
        plt.scatter(positions[idx], velocities[idx], s=2, alpha=0.7)
        
        # Calculate temperature
        kinetic_energies = 0.5 * RB87_MASS * velocities[idx]**2
        temperature = np.mean(kinetic_energies) / k_B
        
        plt.title(f'Time: {t*1e3:.2f} ms, Temperature: {temperature/uK_to_K:.2f} μK')
        plt.xlabel('Position (m)')
        plt.ylabel('Velocity (m/s)')
        plt.grid(True)
        
        # Plot velocity distribution histogram
        plt.subplot(2, 1, 2)
        plt.hist(velocities[idx], bins=30, alpha=0.7)
        plt.xlabel('Velocity (m/s)')
        plt.ylabel('Number of atoms')
        plt.title('Velocity Distribution')
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()
    
    # Create interactive slider
    interact(update_plot, time_idx=widgets.IntSlider(
        min=0, max=100, step=1, value=0,
        description='Time (%):',
        style={'description_width': 'initial'}
    ))

# Create interactive visualization
interactive_visualization(positions, velocities, sisyphus_times)

## 8. Analysis and Discussion

Let's analyze the results of our simulations and discuss the key principles of Sisyphus cooling.

### 8.1 Temperature Evolution Analysis

The temperature evolution plot shows that Sisyphus cooling can achieve significantly lower temperatures than Doppler cooling. This is because Sisyphus cooling is a sub-Doppler cooling technique that can overcome the Doppler cooling limit.

In Doppler cooling, the temperature is limited by the random recoil kicks from spontaneous emission, which leads to a minimum temperature of $T_D = \hbar\Gamma/(2k_B) \approx 146 \, \mu\text{K}$ for Rubidium-87.

In contrast, Sisyphus cooling can reach temperatures close to the recoil limit, which is $T_r = \hbar^2 k^2 / (2m k_B) \approx 0.18 \, \mu\text{K}$ for Rubidium-87. This is achieved through the Sisyphus effect, where atoms lose energy by climbing potential hills and then being optically pumped to lower potential energy states.

### 8.2 Spatial Distribution Analysis

The spatial distribution plots show how atoms become localized in the optical lattice potential wells as they cool. Initially, atoms are distributed randomly with a thermal velocity distribution. As cooling progresses, atoms become trapped in the potential wells, with their positions corresponding to the minima of the potential for their particular ground state sublevel.

This localization is a key feature of Sisyphus cooling and is essential for applications like optical lattice clocks and quantum simulation with ultracold atoms.

### 8.3 Key Principles of Sisyphus Cooling

The key principles of Sisyphus cooling are:

1. **Polarization Gradient**: Counter-propagating laser beams with orthogonal polarizations create a standing wave with a spatially varying polarization (alternating between σ+, linear, and σ- polarization).

2. **State-Dependent Potential**: Different ground state magnetic sublevels experience different light shifts (AC Stark shifts) depending on the local polarization, creating a periodic potential landscape.

3. **Optical Pumping**: As atoms move through this landscape, they undergo optical pumping between different ground state sublevels at positions where the polarization favors specific transitions.

4. **Energy Loss Mechanism**: When an atom moves up a potential hill in one sublevel and is then optically pumped to another sublevel with a lower potential energy, it loses kinetic energy. This process repeats, continuously removing energy from the atom's motion.

### 8.4 Limitations and Future Improvements

Our model has several limitations and could be improved in the following ways:

1. **Simplified Quantum Structure**: We've used a simplified representation of the Rubidium-87 energy levels, focusing only on the F=2 → F'=3 transition and neglecting other hyperfine levels.

2. **One-Dimensional Model**: Our simulation is limited to one spatial dimension, whereas real experiments are three-dimensional.

3. **Semi-Classical Approach**: We've treated atomic motion classically while using quantum mechanics for internal states, which is an approximation.

4. **Damping Coefficient**: We've included a velocity damping coefficient to ensure cooling dominates over heating, which is a simplification of the actual cooling mechanism.

Future improvements could include:

1. **Full Quantum Treatment**: Implement a fully quantum mechanical treatment of both internal states and atomic motion.

2. **Three-Dimensional Model**: Extend the model to three spatial dimensions for more realistic simulations.

3. **Additional Hyperfine Levels**: Include other hyperfine levels and transitions for a more complete model.

4. **Quantum Monte Carlo Approach**: Implement a quantum Monte Carlo wave function approach for more accurate modeling of spontaneous emission.

## 9. Conclusion

In this notebook, we've explored Sisyphus cooling, a sub-Doppler laser cooling technique that allows atoms to reach temperatures below the Doppler cooling limit. We've implemented a simplified educational model using QuTiP and demonstrated the key principles of Sisyphus cooling.

We've seen that Sisyphus cooling can achieve significantly lower temperatures than Doppler cooling, approaching the recoil limit. This makes it a powerful technique for preparing ultracold atoms for applications in quantum information processing, precision metrology, and quantum simulation.

The model we've developed provides a foundation for understanding more complex laser cooling techniques and can be extended to study other quantum optical phenomena.

## 10. References

1. Dalibard, J.; Cohen-Tannoudji, C. (1989). "Laser cooling below the Doppler limit by polarization gradients: simple theoretical models". Journal of the Optical Society of America B. 6 (11): 2023.

2. Steck, D.A. (2001). "Rubidium 87 D Line Data". Available at: https://steck.us/alkalidata/rubidium87numbers.1.6.pdf

3. Metcalf, H.J.; van der Straten, P. (1999). "Laser Cooling and Trapping". Springer.

4. Foot, C.J. (2005). "Atomic Physics". Oxford University Press. Section 9.6.

## 11. Exercises

Here are some exercises to deepen your understanding of Sisyphus cooling:

1. **Parameter Exploration**: Modify the detuning and Rabi frequency parameters and observe how they affect the cooling efficiency.

2. **Recoil Limit**: Investigate how close the Sisyphus cooling can get to the recoil limit by running longer simulations.

3. **Optical Lattice Depth**: Explore how the depth of the optical lattice potential affects the cooling dynamics and final temperature.

4. **Initial Temperature**: Start with different initial temperatures and observe how the cooling time changes.

5. **Visualization**: Create an animation of atoms moving in the optical lattice potential during the cooling process.